In [1]:
# You can ignore this part of code as I'm using drive to access segment.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)


In [0]:
import tensorflow as tf
import cv2

tf.enable_eager_execution()

from waymo_open_dataset import dataset_pb2 as open_dataset

In [0]:
FILENAME = '/content/drive/My Drive/Waymo/training_0024/segment-5083516879091912247_3600_000_3620_000.tfrecord'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
frames = []
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    frames.append(frame)


In [5]:
print(len(frames))

197


In [0]:
cframe = frames[1]

In [7]:
dir(cframe)

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FindInitializationErrors',
 'FromString',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WhichOneof',
 '_CheckCalledFromGeneratedFile',
 '_SetListener',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_extensions_by_name',
 '_extensions_by_number',
 'camera_labels',
 'context',
 'images',
 'laser_labels',
 'lasers',


In [0]:
# template = {'id': None, 'name': '', 'info': None}
# info = {'position': '', 'velocity': None}
camera_names = {0: 'UNKNOWN', 1: 'FRONT',  2: 'FRONT_LEFT', 3: 'FRONT_RIGHT', 4: 'SIDE_LEFT', 5: 'SIDE_RIGHT'}

In [0]:
from collections import defaultdict
import numpy as np

vel_info = []

fcount = 0

for frame in frames:
    template = {'id': None, 'name': '', 'info': None}
    template['id'] = fcount
    name = frame.context.name
    template['name'] = name + '_' + str(frame.timestamp_micros)
    v_dict = defaultdict()
    for img in frame.images:
        img_name = img.name
        velocity = img.velocity
        vel_vec = np.array([velocity.v_x, velocity.v_y, velocity.v_z])
        v_dict[img_name] = vel_vec
    template['info'] = v_dict
    vel_info.append(template)
    fcount += 1

In [10]:
# Velocity per frame for each of the 5 cameras.
vel_info[156]

{'id': 156,
 'info': defaultdict(None,
             {1: array([ 0.0855178 , -0.67239869, -0.00629936]),
              2: array([ 0.09465287, -0.65490174, -0.00618541]),
              3: array([ 0.11702001, -0.67823714, -0.00460137]),
              4: array([ 0.08380947, -0.64980018, -0.00662386]),
              5: array([ 0.08867761, -0.69984674, -0.00491398])}),
 'name': '5083516879091912247_3600_000_3620_000_1546882660072777'}

In [0]:
def get_avg_vel(vels):
    v = np.array([0, 0, 0])
    for vel in vels:
        v = v + vel
    return v / len(vels)

def chunkIt(seq, num):
    avg = round(len(seq) / float(num))
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

In [0]:
chunks = chunkIt(vel_info, 20)

In [0]:
avg_vel = []
count = 0
for chunk in chunks:
    template = {'id': None, 'info': None}
    template['id'] = count
    v_avg_dict = defaultdict()
    for i in range(1, 6):
        vs = []
        for vel in chunk:
            vs.append(vel['info'][i])
        v_avg = get_avg_vel(vs)
        v_avg_dict[i] = v_avg
    template['info'] = v_avg_dict
    count += 1
    avg_vel.append(template)

In [14]:
avg_vel[0:2]

[{'id': 0,
  'info': defaultdict(None,
              {1: array([ 1.10203767, -6.22341208, -0.06664711]),
               2: array([ 1.10499606, -6.22440891, -0.06694557]),
               3: array([ 1.10273607, -6.2227354 , -0.06774748]),
               4: array([ 1.10432649, -6.22349725, -0.06593632]),
               5: array([ 1.1037825 , -6.22007484, -0.06698141])})},
 {'id': 1,
  'info': defaultdict(None,
              {1: array([ 1.01769484, -6.08965421, -0.0681729 ]),
               2: array([ 1.0180984 , -6.0931828 , -0.06880801]),
               3: array([ 1.0156178 , -6.08479257, -0.06755502]),
               4: array([ 1.02248808, -6.09567194, -0.06957604]),
               5: array([ 1.01179196, -6.08609352, -0.06874494])})}]

In [0]:
# Acceleration per second.
acc_info = []
idict = defaultdict()
for i in range(1, 6):
    idict[i] = np.array([0.0, 0.0, 0.0])
t = {'id': 0, 'info': idict}
acc_info.append(t)

i = 0
while i < 19:
    acc_dict = defaultdict()
    template = {'id': None, 'info': None}
    template['id'] = i + 1
    for j in range(1, 6):
        a_1 = avg_vel[i]['info'][j]
        a_2 = avg_vel[i+1]['info'][j]
        acc = a_2 - a_1
        acc_dict[j] = acc
    template['info'] = acc_dict
    acc_info.append(template)
    i += 1


In [16]:
acc_info

[{'id': 0,
  'info': defaultdict(None,
              {1: array([0., 0., 0.]),
               2: array([0., 0., 0.]),
               3: array([0., 0., 0.]),
               4: array([0., 0., 0.]),
               5: array([0., 0., 0.])})},
 {'id': 1,
  'info': defaultdict(None,
              {1: array([-0.08434283,  0.13375788, -0.00152578]),
               2: array([-0.08689767,  0.13122611, -0.00186244]),
               3: array([-0.08711827,  0.13794284,  0.00019246]),
               4: array([-0.08183841,  0.12782531, -0.00363971]),
               5: array([-0.09199054,  0.13398132, -0.00176353])})},
 {'id': 2,
  'info': defaultdict(None,
              {1: array([-0.12038853,  0.35427222, -0.00411951]),
               2: array([-0.11688848,  0.35248823, -0.00289387]),
               3: array([-0.12143987,  0.35982108, -0.00444277]),
               4: array([-0.11884255,  0.34115028, -0.00306966]),
               5: array([-0.11350417,  0.37049584, -0.00380494])})},
 {'id': 3,
  'info'

In [0]:
# Acceleration per frame.
frame_acc = []
k = 0
l = 0
for i in range(len(frames)):
    template = {'id': None, 'info': None}
    template['id'] = i
    template['info'] = acc_info[l]['info']
    frame_acc.append(template)
    k += 1
    if k == 10:
        l += 1
        k = 0

In [18]:
frame_acc

[{'id': 0,
  'info': defaultdict(None,
              {1: array([0., 0., 0.]),
               2: array([0., 0., 0.]),
               3: array([0., 0., 0.]),
               4: array([0., 0., 0.]),
               5: array([0., 0., 0.])})},
 {'id': 1,
  'info': defaultdict(None,
              {1: array([0., 0., 0.]),
               2: array([0., 0., 0.]),
               3: array([0., 0., 0.]),
               4: array([0., 0., 0.]),
               5: array([0., 0., 0.])})},
 {'id': 2,
  'info': defaultdict(None,
              {1: array([0., 0., 0.]),
               2: array([0., 0., 0.]),
               3: array([0., 0., 0.]),
               4: array([0., 0., 0.]),
               5: array([0., 0., 0.])})},
 {'id': 3,
  'info': defaultdict(None,
              {1: array([0., 0., 0.]),
               2: array([0., 0., 0.]),
               3: array([0., 0., 0.]),
               4: array([0., 0., 0.]),
               5: array([0., 0., 0.])})},
 {'id': 4,
  'info': defaultdict(None,
             

In [19]:
# Bounding box position and speed of those objects. Already given in dataset
cframe.projected_lidar_labels

[name: FRONT
labels {
  box {
    center_x: 900.0707373235039
    center_y: 643.2308554712795
    width: 46.7768949177522
    length: 66.69637415572697
  }
  metadata {
    speed_x: -1.1664840805236114
    speed_y: 7.84626482302943
    accel_x: -0.008816161555274649
    accel_y: 0.06387179182016565
  }
  type: TYPE_VEHICLE
  id: "1Um-rXnwNbPj2B3Yl_D66g_FRONT"
}
labels {
  box {
    center_x: 1036.0602886542106
    center_y: 821.8791627330324
    width: 282.2025766232649
    length: 357.0193880317172
  }
  metadata {
    speed_x: 0.9683168568045039
    speed_y: -6.358318974118063
    accel_x: 0.00013001751280443863
    accel_y: -0.03844931994699009
  }
  type: TYPE_VEHICLE
  id: "62AdzV8pml3Vh3QK5lmC_w_FRONT"
}
labels {
  box {
    center_x: 787.2674465546751
    center_y: 690.2309647545051
    width: 102.70380649062247
    length: 163.2872704756071
  }
  metadata {
    speed_x: -1.453143485604665
    speed_y: 9.7033168027746
    accel_x: -0.06752425920335034
    accel_y: 0.598810404501